In [1]:
%load_ext sparkmagic.magics
from dsx_core_utils import proxy_util,dsxhi_util
proxy_util.configure_proxy_livy()
dsxhi_util.list_livy_endpoints()

success configuring sparkmagic livy.
['https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1', 'https://qlawsdl001038a.ad.datalake.foc.zone:8443/gateway/dsx/livy/v1']


In [2]:
%%spark config
{"executorCores": 4, "numExecutors": 5, "executorMemory": "10g", 
 "driverMemory": "8g", "proxyUser": "jchen-", "driverCores": 1, 
 "conf": {"spark.yarn.appMasterEnv.THEANO_FLAGS": "base_compiledir=${PWD}/.theano"}}

In [3]:
%spark add -s refimodel -k -l python -u https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
22879,application_1590030838276_68046,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [4]:
%%spark

print(spark.version)

2.3.0.2.6.5.0-292

In [5]:
%%spark

import pyspark
import os, sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.functions import length, col, when, count, countDistinct, mean, lit, sum, udf, desc, min, max
from pyspark.sql import DataFrameStatFunctions as statFunc
from pyspark.sql.types import *

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier

from pyspark.ml.feature import VectorIndexer, VectorAssembler, VectorSlicer, StringIndexer, QuantileDiscretizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import os
import datetime
import time
import pandas as pd

spark = SparkSession.builder.getOrCreate()

In [6]:
%%spark

model_save_path = '/dev/projects/retention_models/refi_payoff/models/'
training_data_path = '/dev/projects/retention_models/refi_payoff/training/data/train'
test_data_path = '/dev/projects/retention_models/refi_payoff/training/data/test'
result_path = '/dev/projects/retention_models/refi_payoff/training/results/'

monthly_prep_path = '/dev/projects/retention_models/monthly_snapshot/monthly_preprocessed/'
actual_path = '/dev/projects/retention_models/actual_value/'

# 1. Load Data

### Preprocessed Data

In [7]:
%%spark

df201612 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201612.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201703 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201703.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201706 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201706.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201709 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201709.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201712 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201712.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201803 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201803.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201806 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201806.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201809 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201809.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201812 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201812.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201903 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201903.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201906 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201906.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201909 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201909.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201912 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201912.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202003 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202003.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')


df_preprocessed = df201612.union(df201703).union(df201706).union(df201709).union(df201712)\
                            .union(df201803).union(df201806).union(df201809).union(df201812)\
                            .union(df201903).union(df201906).union(df201909).union(df201912)\
                            .union(df202003)\
                            .drop('borrowerage_bucket','p_married_exp','p_31_50_exp')

In [8]:
%%spark

target = 'refipayoff'

### Actual Payoff Data

In [9]:
%%spark

actual = spark.read.parquet(actual_path + 'Actual_payoff_20200610.parquet')\
                .select('servicecalendardate', 'ln_no', 'refipayoff')

actual.show(1)

+-------------------+----------+----------+
|servicecalendardate|     ln_no|refipayoff|
+-------------------+----------+----------+
|         2016-10-31|3219834719|       0.0|
+-------------------+----------+----------+
only showing top 1 row

### Join actual data with preprocessed data

In [10]:
%%spark

df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()
df_all.groupby('servicecalendardate').agg(count('ln_no'), sum(target)).orderBy(col('servicecalendardate').desc()).show()

+-------------------+------------+---------------+
|servicecalendardate|count(ln_no)|sum(refipayoff)|
+-------------------+------------+---------------+
|         2020-03-31|     1867297|        62522.0|
|         2019-12-31|     1802243|       104715.0|
|         2019-09-30|     1763391|        95750.0|
|         2019-06-30|     1822001|        78836.0|
|         2018-12-31|     1726017|        38086.0|
|         2018-09-30|     1683583|        27599.0|
|         2018-06-30|     1619695|        31482.0|
|         2018-03-31|     1577779|        28882.0|
|         2017-12-31|     1530004|        29156.0|
|         2017-09-30|     1468441|        38311.0|
|         2017-06-30|     1423608|        42331.0|
|         2017-03-31|     1379094|        31566.0|
|         2016-12-31|     1326375|        26235.0|
+-------------------+------------+---------------+

# 2. Build Models

### Sample Data

In [11]:
%%spark

# create fundtion for undersampling
def data_split(df, train_months, target, rate):
    
    ## Split Train/Validation From Test ## 
    train_validate = df.where(col('servicecalendardate').isin(train_months))
    
    ## Split Out Payoffs into Train/Validate ##
    train_refi_all = train_validate.filter(train_validate[target] == '1.')
    train_refi, validate_refi = train_refi_all.randomSplit([0.7, 0.3], seed=123)

    ## Split Out Non-Payoffs to Train/Validate
    train_nonrefi_all = train_validate.filter(train_validate[target] == '0.')
    train_nonrefi, validate_nonrefi = train_nonrefi_all.randomSplit([0.7, 0.3], seed=123)

    ## Undersample Non-Payoffs ## 
    train_non, notused1 = train_nonrefi.randomSplit([rate, 1-rate], seed=123)
    val_non, notused2 = validate_nonrefi.randomSplit([rate, 1-rate], seed=123)

    ## Create Final Training/Validate Sets
    train = train_refi.unionAll(train_non)
    validate = validate_refi.unionAll(val_non)
    train.cache()
    
    return train, validate

In [12]:
%%spark

train_months = ['2017-12-31', '2018-03-31', '2018-06-30', '2018-09-30', '2018-12-31', '2019-03-31', '2019-06-30']

df_train, df_validate = data_split(df_all, train_months, target, 0.05)

In [13]:
%%spark

df_train.groupby(['servicecalendardate', target]).count().orderBy('servicecalendardate', target).show()

+-------------------+----------+-----+
|servicecalendardate|refipayoff|count|
+-------------------+----------+-----+
|         2017-12-31|       0.0|52380|
|         2017-12-31|       1.0|20430|
|         2018-03-31|       0.0|54101|
|         2018-03-31|       1.0|20368|
|         2018-06-30|       0.0|55490|
|         2018-06-30|       1.0|22022|
|         2018-09-30|       0.0|58094|
|         2018-09-30|       1.0|19246|
|         2018-12-31|       0.0|59376|
|         2018-12-31|       1.0|26757|
|         2019-06-30|       0.0|61442|
|         2019-06-30|       1.0|55378|
+-------------------+----------+-----+

### Creat Pipeline

In [24]:
%%spark

def create_pipeline(target, *arg):
    
    exclude_cols = ('servicecalendardate', 'ln_no', 'og_note_dt', 'mosaic_group_pur_exp', 'ratespread_min_pur_exp',
                'ln_purpose_type', 'investornameshort', 'ln_ann_int_rt', target) 
    cat_cols = [i[0] for i in df_train.dtypes if ((i[1]=='string') & (~i[0].endswith(exclude_cols)))]
    num_cols = [i[0] for i in df_train.dtypes if ((i[1].startswith(('int', 'double'))) & (~i[0].endswith(exclude_cols)))]
    
    stages = []
    
    for col in cat_cols:
        
        #Category indexing with StringIndexer
        indexer = StringIndexer(inputCol = col, outputCol = col+'_idx').setHandleInvalid('keep')
        stages += [indexer]
        
    #assemblerInputs = [c+'_vec' for c in cat_cols] + num_cols
    assemblerInputs = [c+'_idx' for c in cat_cols] + num_cols
    assembler = VectorAssembler(inputCols = assemblerInputs, outputCol = 'vectFeatures')
    
    stages += [assembler]
    
    lr = LogisticRegression(maxIter=100, regParam=0.1, elasticNetParam=0.0, fitIntercept = True,
                            featuresCol='vectFeatures', labelCol=target)
    rf = RandomForestClassifier(numTrees=250, maxDepth = 5, featuresCol='vectFeatures', labelCol=target)
    gbt = GBTClassifier(maxIter=100, featuresCol='vectFeatures', labelCol=target)

    pipeline_lr = Pipeline(stages = stages + [lr])
    pipeline_rf = Pipeline(stages = stages + [rf])
    pipeline_gbt = Pipeline(stages = stages + [gbt])
 
    if "lr" in arg and "rf" in arg and "gbt" in arg:
        return lr, rf, gbt, pipeline_lr, pipeline_rf, pipeline_gbt
    elif "lr" in arg and "rf" in arg:
        return lr, rf, pipeline_lr, pipeline_rf
    elif "lr" in arg and "gbt" in arg:
        return lr, gbt, pipeline_lr, pipeline_gbt 
    elif "rf" in arg and "gbt" in arg:
        return rf, gbt, pipeline_rf, pipeline_gbt
    elif "rf" in arg:
        return rf, pipeline_rf
    elif "lr" in arg:
        return lr, pipeline_lr
    elif "gbt" in arg:
        return gbt, pipeline_gbt
    else:
        return gbt, pipeline_gbt

In [25]:
%%spark

def fit_pipeline(pipeline, training_dataset):
    model_pipeline = pipeline.fit(training_dataset)
    return model_pipeline
    
def persist_modelpersist_(pipeline_model, model_name, model_save_path=model_save_path):
    pipeline_model.save(model_save_path + model_name)

In [26]:
%%spark

lr, rf, gbt, pipeline_lr, pipeline_rf, pipeline_gbt = create_pipeline(target, "lr", "rf", "gbt")

In [27]:
%%spark

# Logistic Regression
start = time.time()
model_pipeline_lr = fit_pipeline(pipeline_lr, df_train)
end = time.time()

print('Logistic Regression Training Time:', end - start)

Logistic Regression Training Time: 49.82642197608948

In [28]:
%%spark

# Random Forest
start = time.time()
model_pipeline_rf = fit_pipeline(pipeline_rf, df_train)
end = time.time()

print('Random Forest Training Time:', end - start)

Random Forest Training Time: 61.7143292427063

In [29]:
%%spark

# Gradient Boosting
start = time.time()
model_pipeline_gbt = fit_pipeline(pipeline_gbt, df_train)
end = time.time()

print('Gradient Boosting Training Time:', end - start)

Gradient Boosting Training Time: 723.8272330760956

### Save Models

In [30]:
%%spark

persist_modelpersist_(model_pipeline_lr, 'refiV1_4_LR_20200513')
persist_modelpersist_(model_pipeline_rf, 'refiV1_4_RF_20200513')
persist_modelpersist_(model_pipeline_gbt, 'refiV1_4_GBT_20200513')

### Read Models

In [10]:
%%spark

model_pipeline_lr = PipelineModel.load(path = model_save_path+'refiV1_4_LR_20200513')
model_pipeline_rf = PipelineModel.load(path = model_save_path+'refiV1_4_RF_20200513')
model_pipeline_gbt = PipelineModel.load(path = model_save_path+'refiV1_4_GBT_20200513')

# 3. Variable Importance

In [15]:
%%spark
def extractFeatureImp(model_pipeline, df_train, featuresCol):
    
    featureImp = model_pipeline.stages[-1].featureImportances
    transformed = model_pipeline.transform(df_train)
    list_extract = []
    
    for i in transformed.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + transformed.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
        
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    
    return(varlist.sort_values('score', ascending = False))
    return varlist

In [16]:
%%spark

# GBT
gbt_feature_importance = extractFeatureImp(model_pipeline_gbt, df_train, "vectFeatures")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(gbt_feature_importance)

    idx                         name  \
5    13                 ageinmon_exp   
7    15              currentcltv_exp   
4    12           ratespread_min_exp   
6    14                og_mtg_am_exp   
8    16                orig_fico_exp   
20    6      ln_purpose_type_exp_idx   
18    4  loantypedescription_exp_idx   
13   21                    ln_tr_exp   
1     9               home_value_exp   
21    7  og_occupy_stat_type_exp_idx   
0     8                   ct_age_exp   
19    5     loanamortizationtype_idx   
10   18           LiveYears_long_exp   
17    3    mosaic_group_refi_exp_idx   
16    2                 ct_3_exp_idx   
3    11       personnum_per_room_exp   
9    17          LiveYears_short_exp   
12   20         issingleborrower_exp   
15    1                 ct_2_exp_idx   
2    10                 p_edu_hs_exp   
14    0                 ct_1_exp_idx   
11   19            LiveYears_grp_exp   

                                                 vals     score  
5            

# 4. ROC

### Predictions

In [11]:
%%spark

def predict(model_pipeline, data):
    
    _scoreUdf = udf(lambda v: float(v[1]), DoubleType())
    
    prediction = model_pipeline.transform(data)
    pred_df = prediction.withColumn('pred', _scoreUdf(prediction['probability']))
    return pred_df

### ROC

In [18]:
%%spark

def roc_auc(df, target):   
    
    pred_df_lr = predict(model_pipeline_lr, df)
    pred_df_rf = predict(model_pipeline_rf, df)
    pred_df_gbt = predict(model_pipeline_gbt, df)

    ### Calculate ROC ###  
    evalPred_lr = pred_df_lr.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')
    evalPred_rf = pred_df_rf.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')
    evalPred_gbt = pred_df_gbt.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')

    evaluatorLR = BinaryClassificationEvaluator()
    evaluatorRF = BinaryClassificationEvaluator()
    evaluatorGBT = BinaryClassificationEvaluator()

    print("Test Area Under ROC - LR: " + str(evaluatorLR.evaluate(evalPred_lr, {evaluatorLR.metricName: "areaUnderROC"})))        
    print("Test Area Under ROC - RF: " + str(evaluatorRF.evaluate(evalPred_rf, {evaluatorRF.metricName: "areaUnderROC"})))        
    print("Test Area Under ROC - GBT: " + str(evaluatorGBT.evaluate(evalPred_gbt, {evaluatorGBT.metricName: "areaUnderROC"})))
    print("\n")

In [19]:
%%spark

test_months = ['2017-09-30', '2019-09-30', '2019-12-31']
df_test = df_all.where(col('servicecalendardate').isin(test_months))

for df in [df_train, df_validate, df_test]:
    roc_auc(df, target)

Test Area Under ROC - LR: 0.6332322703577062
Test Area Under ROC - RF: 0.680734951367731
Test Area Under ROC - GBT: 0.7252698287642437


Test Area Under ROC - LR: 0.6343274599427369
Test Area Under ROC - RF: 0.6792016999576734
Test Area Under ROC - GBT: 0.7196060082250103


Test Area Under ROC - LR: 0.6107240826340268
Test Area Under ROC - RF: 0.6768353729994416
Test Area Under ROC - GBT: 0.708076291895596

In [13]:
%%spark

test_months = ['2017-09-30', '2019-09-30', '2019-12-31']
df_test = df_all.where(col('servicecalendardate').isin(test_months))


# 5. Capture Rate

In [12]:
%%spark

def deciles(df, model, target):
    
    pred_df = predict(model, df)

    pred_df = QuantileDiscretizer(numBuckets=10, inputCol="pred", outputCol="decile", relativeError=0.00001,
                             handleInvalid="error").fit(pred_df).transform(pred_df)
    pred_df = pred_df.withColumn('decile', (10 - F.col('decile')).cast('int'))
    
    window_cumsum = Window.orderBy('decile').rangeBetween(Window.unboundedPreceding, 0)
    total_target = pred_df.select(F.sum(target)).collect()[0][0]
    df_out = pred_df\
        .groupBy('decile', )\
        .agg(F.count('ln_no').alias('decile_cnt'), F.sum(target).alias('payoff_cnt'))\
        .withColumn('cum_sum', F.sum('payoff_cnt').over(window_cumsum) / total_target)\
        .sort('decile')

    return df_out

In [45]:
%%spark

df_train.groupBy('servicecalendardate').count().show()

+-------------------+------+
|servicecalendardate| count|
+-------------------+------+
|         2018-06-30| 77512|
|         2018-09-30| 77340|
|         2017-12-31| 72810|
|         2018-12-31| 86133|
|         2019-06-30|116820|
|         2018-03-31| 74469|
+-------------------+------+

In [30]:
%%spark

date_label = test_months[2]
print(date_label)

2019-12-31

In [24]:
%%spark
#print(date_label)
#deciles(df_all.filter(col('servicecalendardate') == date_label), model_pipeline_lr, target).show()

In [26]:
%%spark
#print(date_label)
#deciles(df_all.filter(col('servicecalendardate') == date_label), model_pipeline_rf, target).show()

In [31]:
%%spark
print(date_label)
deciles(df_all.filter(col('servicecalendardate') == date_label), model_pipeline_gbt, target).show()

2019-12-31
+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    180229|   25866.0|0.24701332187365707|
|     2|    180210|   17371.0| 0.4129016855273839|
|     3|    180207|   13264.0| 0.5395693071670725|
|     4|    180210|   11468.0| 0.6490856133314233|
|     5|    180246|    9632.0| 0.7410686148116316|
|     6|    180236|    8537.0|  0.822594661700807|
|     7|    180205|    7373.0| 0.8930048226137611|
|     8|    180240|    5954.0| 0.9498639163443633|
|     9|    180230|    3959.0| 0.9876712982858235|
|    10|    180230|    1291.0|                1.0|
+------+----------+----------+-------------------+

In [32]:
%%spark

actual1 = spark.read.parquet(actual_path + 'Actual_payoff_20200505.parquet')\
                .select('servicecalendardate', 'ln_no', 'refipayoff')

df_all1 = df_preprocessed.join(actual1, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates().where(col('servicecalendardate') == '2019-12-31')

deciles(df_all1.filter(col('servicecalendardate') == date_label), model_pipeline_gbt, target).show()

+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    180219|   26262.0| 0.2452192425487413|
|     2|    180220|   17688.0|0.41037947262269364|
|     3|    180210|   13548.0|  0.536882796743109|
|     4|    180227|   11711.0| 0.6462332860237544|
|     5|    180237|    9875.0| 0.7384402778815269|
|     6|    180213|    8778.0| 0.8204041234033017|
|     7|    180229|    7604.0| 0.8914058414880107|
|     8|    180237|    6172.0| 0.9490363785762307|
|     9|    180214|    4091.0| 0.9872357511018152|
|    10|    180237|    1367.0|                1.0|
+------+----------+----------+-------------------+

# 6. Predictions

In [13]:
%%spark

def fullMonthPrediction(df):
    
    pred_df_lr = predict(model_pipeline_lr, df)
    validationPredictionsLR = pred_df_lr.select('ln_no', 'pred')\
                                            .withColumnRenamed('pred', 'logRegProb')
    
    pred_df_rf = predict(model_pipeline_rf, df)
    validationPredictionsRF = pred_df_rf.select('ln_no', 'pred')\
                                            .withColumnRenamed('pred', 'randForProb')
    
    pred_df_gbt = predict(model_pipeline_gbt, df)
    validationPredictionsGBT = pred_df_gbt.select('ln_no', 'pred')\
                                            .withColumnRenamed('pred', 'gbtProb')
    
    combinedFinalPred = validationPredictionsLR.join(validationPredictionsRF, on='ln_no', how='left')\
                                                .join(validationPredictionsGBT, on='ln_no', how='left')\
                                                .dropDuplicates()
    
    return combinedFinalPred

In [14]:
%%spark

def save_predictions_csv(df_pred, filename, result_path = result_path):
    
    df_pred.coalesce(1).write.csv(result_path + filename, header=True)

In [15]:
%%spark

def pred_save(df, servicedate, filename, result_path = result_path):
    
    df1 = df.where(col('servicecalendardate') == servicedate)
    df_pred = fullMonthPrediction(df1)
    
    save_predictions_csv(df_pred, filename)

In [16]:
%%spark

def pred_save_newmonth(preprocessed_file, filename, monthly_prep_path=monthly_prep_path, result_path=result_path):
    
    df = spark.read.parquet(monthly_prep_path + preprocessed_file)
    df_pred = fullMonthPrediction(df)
    
    print(df.count())
    save_predictions_csv(df_pred, filename)

In [18]:
%%spark

# 2017-9
pred_save(df_test.where(col('servicecalendardate') == '2017-09-30'), '2017-09-30', 'pred_refiV1_4_sep17_20200514.csv')


# 2019-9
pred_save(df_test.where(col('servicecalendardate') == '2019-09-30'), '2019-09-30', 'pred_refiV1_4_sep19_20200514.csv')

          
# 2019-12
pred_save(df_test.where(col('servicecalendardate') == '2019-12-31'), '2019-12-31', 'pred_refiV1_4_dec19_20200514.csv')
       

In [19]:
%%spark

print(df_all.where(col('servicecalendardate') == '2017-09-30').count())
print(spark.read.csv(result_path + 'pred_refiV1_4_sep17_20200514.csv', header = True).count())


print(df_all.where(col('servicecalendardate') == '2019-09-30').count())
print(spark.read.csv(result_path + 'pred_refiV1_4_sep19_20200514.csv', header = True).count())


print(df_all.where(col('servicecalendardate') == '2019-12-31').count())
print(spark.read.csv(result_path + 'pred_refiV1_4_dec19_20200514.csv', header = True).count())



1468441
1468441
1763391
1763391
1802243
1802243

In [17]:
%%spark

df202003 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202003.parquet')
df_preprocessed = df202003.drop('borrowerage_bucket','p_married_exp','p_31_50_exp')
df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()
df_all.groupby('servicecalendardate').agg(count('ln_no'), sum(target)).orderBy(col('servicecalendardate').desc()).show()

+-------------------+------------+---------------+
|servicecalendardate|count(ln_no)|sum(refipayoff)|
+-------------------+------------+---------------+
|         2020-03-31|     1867297|        62522.0|
+-------------------+------------+---------------+

In [22]:
%%spark

# 2020-03
pred_save(df_all.where(col('servicecalendardate') == '2020-03-31'), '2020-03-31', 'pred_refiV1_4_mar20_20200514.csv')

In [23]:
%%spark

print(spark.read.csv(result_path + 'pred_refiV1_4_mar20_20200514.csv', header = True).count())

1867297

In [25]:
%%spark

deciles(df_all.filter(col('servicecalendardate') == '2020-03-31'), model_pipeline_gbt, target).show()

+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    186704|   13777.0|0.22035443523879594|
|     2|    186751|   10976.0|0.39590864015866417|
|     3|    186740|    8277.0| 0.5282940404977448|
|     4|    186715|    6727.0| 0.6358881673650875|
|     5|    186717|    6492.0| 0.7397236172867151|
|     6|    186742|    5759.0| 0.8318351940117079|
|     7|    186735|    4741.0| 0.9076645020952625|
|     8|    186715|    3187.0|  0.958638559227152|
|     9|    186715|    2023.0| 0.9909951697002655|
|    10|    186763|     563.0|                1.0|
+------+----------+----------+-------------------+

In [26]:
%%spark

df202004 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202004.parquet')
df_preprocessed = df202004.drop('borrowerage_bucket','p_married_exp','p_31_50_exp')
df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()

pred_save(df_all.where(col('servicecalendardate') == '2020-04-30'), '2020-04-30', 'pred_refiV1_4_apr20_20200514.csv')
print(spark.read.csv(result_path + 'pred_refiV1_4_apr20_20200514.csv', header = True).count())

deciles(df_all.filter(col('servicecalendardate') == '2020-04-30'), model_pipeline_gbt, target).show()

DataFrame[servicecalendardate: date, count(ln_no): bigint, sum(refipayoff): double]
1899099
+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    189904|    4044.0|0.23578800069966765|
|     2|    189889|    2937.0| 0.4070316599615183|
|     3|    189936|    2329.0| 0.5428254912250015|
|     4|    189899|    1907.0| 0.6540143431869861|
|     5|    189909|    1734.0| 0.7551163197481197|
|     6|    189901|    1503.0| 0.8427496938953997|
|     7|    189927|    1232.0| 0.9145822401026179|
|     8|    189906|     840.0|  0.963558976152994|
|     9|    189897|     519.0| 0.9938196023555478|
|    10|    189931|     106.0|                1.0|
+------+----------+----------+-------------------+

In [28]:
%%spark

df_all.groupby('servicecalendardate').agg(count('ln_no'), sum(target)).orderBy(col('servicecalendardate').desc()).show()

+-------------------+------------+---------------+
|servicecalendardate|count(ln_no)|sum(refipayoff)|
+-------------------+------------+---------------+
|         2020-04-30|     1899099|        17151.0|
+-------------------+------------+---------------+

In [41]:
%%spark

df202002 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202002.parquet')
df_preprocessed = df202002.drop('borrowerage_bucket','p_married_exp','p_31_50_exp')
df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()
pred_save(df_all.where(col('servicecalendardate') == '2020-02-29'), '2020-02-29', 'pred_refiV1_4_feb20_20200514.csv')

deciles(df_all.filter(col('servicecalendardate') == '2020-02-29'), model_pipeline_gbt, target).show()

+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    184673|   24445.0|0.23387644588168885|
|     2|    184677|   17463.0| 0.4009529185522527|
|     3|    184698|   13762.0| 0.5326202389950345|
|     4|    184681|   12389.0| 0.6511514432506387|
|     5|    184695|   10712.0| 0.7536380248945188|
|     6|    184689|    8754.0| 0.8373915289750385|
|     7|    184672|    7048.0|  0.904822954238861|
|     8|    184709|    5449.0| 0.9569560184077841|
|     9|    184661|    3506.0| 0.9904995168435051|
|    10|    184709|     993.0|                1.0|
+------+----------+----------+-------------------+

In [42]:
%%spark

df_all = spark.read.csv(result_path + 'pred_refiV1_3_feb20_20200305.csv', header = True)
df_all.printSchema()

root
 |-- ln_no: string (nullable = true)
 |-- og_note_dt: string (nullable = true)
 |-- loantypedescription: string (nullable = true)
 |-- loanamortizationtype: string (nullable = true)
 |-- investornameshort: string (nullable = true)
 |-- loan_term: string (nullable = true)
 |-- loan_age: string (nullable = true)
 |-- currentcltv: string (nullable = true)
 |-- loan_interest_rate: string (nullable = true)
 |-- logRegProb: string (nullable = true)
 |-- randForProb: string (nullable = true)
 |-- gbtProb: string (nullable = true)

In [43]:
%%spark

df = df_all.withColumn('pred', col('gbtProb').cast('double'))\
                .withColumn('servicecalendardate', lit('2020-02-29'))\
                .join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()
pred_df = QuantileDiscretizer(numBuckets=10, inputCol="pred", outputCol="decile", relativeError=0.00001,
                             handleInvalid="error").fit(df).transform(df)
pred_df = pred_df.withColumn('decile', (10 - F.col('decile')).cast('int'))
    
window_cumsum = Window.orderBy('decile').rangeBetween(Window.unboundedPreceding, 0)
total_target = pred_df.select(F.sum(target)).collect()[0][0]
df_out = pred_df\
        .groupBy('decile', )\
        .agg(F.count('ln_no').alias('decile_cnt'), F.sum(target).alias('payoff_cnt'))\
        .withColumn('cum_sum', F.sum('payoff_cnt').over(window_cumsum) / total_target)\
        .sort('decile')
df_out.show()

+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    184678|   21641.0|0.20704930109738712|
|     2|    184685|   16132.0|0.36139149070521714|
|     3|    184684|   13726.0|0.49271438275561846|
|     4|    184685|   12516.0| 0.6124606538398982|
|     5|    184675|   10899.0| 0.7167363496330881|
|     6|    184713|    9263.0|  0.805359688483654|
|     7|    184651|    8106.0| 0.8829134815013251|
|     8|    184714|    6831.0| 0.9482687689555209|
|     9|    184665|    4287.0| 0.9892844500148296|
|    10|    184714|    1120.0|                1.0|
+------+----------+----------+-------------------+